### To test correlations between HEXACO lexical analysis scores and HEXACO-PI-R 100 scores.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np

scores_file = 'data/popc_results.csv'
scores_data = pd.read_csv(scores_file, index_col=0)
to_fix = [x for x in scores_data.index if "'" in x]
print(f'Fixing names: {", ".join(to_fix)}.')
for n in to_fix:
    scores_data.rename(index={n:n.replace("'", '')}, inplace=True)
print(f"Read in {len(scores_data)} responses from {scores_file}")

agent_loadings = pd.read_csv('intermediate/loadings_popc_10.csv', index_col=0)
agent_loadings.columns = ['Dishonesty',
                          'Disagreeableness',
                          'Introversion',
                          'Unconscientiousness',
                          'Unheroic',
                          'Unscholarly',
                          'Gendered-emotionality',
                          'Unsentimentality',
                          'Insensitivity',
                          'Unartistic']

Fixing names: PatrickO'Connor, Margaret'Maggie'Dupont, MartinO'Donnell, Johnathan'BigJohn'Moretti, JohnO'Reilly.
Read in 310 responses from data/popc_results.csv


In [2]:
# calculate score for the agents, based on the recovered factors' loadings.
df_dimensions = pd.DataFrame(index=scores_data.index)
for factor in agent_loadings.columns:
    terms = list(agent_loadings[factor].dropna().index)
    weights = agent_loadings[factor].dropna()
    scores = [x-5 for x in scores_data[terms].values]
    scores = np.array(scores)
    scores = scores * weights.values
    df_dimensions[factor] = scores.sum(axis=1)
    
df_dimensions

,Dishonesty,Disagreeableness,Introversion,Unconscientiousness,Unheroic,Unscholarly,Gendered-emotionality,Unsentimentality,Insensitivity,Unartistic
DaveThomas,-187.801466,157.643697,34.223071,-471.683534,-58.906754,-119.082854,-111.048052,31.585243,-43.596766,5.739249
OliverHarris,-200.566474,190.108360,-62.893161,-396.595915,-114.461764,-103.182884,-26.907505,-73.786820,-97.946922,2.369920
DavidHarrison,-185.919976,-56.309447,-26.633121,-489.013335,38.008888,-190.746573,-97.649288,7.145955,7.086425,-38.082949
JerryMcAlister,299.266586,-57.928586,-278.533774,-247.742093,-327.966392,-106.208720,-81.729967,57.190520,37.768405,-93.513383
GregoryHarrison,81.601540,54.534997,173.808648,-480.600928,-64.183731,-208.497500,-124.469778,92.652522,29.369394,-20.142104
...,...,...,...,...,...,...,...,...,...,...
EmmaJohnson,-228.095558,-338.001805,-209.428340,-54.453986,-155.984677,-108.440796,8.816598,4.777543,-77.297799,-2.434766
AmeliaMartinez,-239.314241,-338.478512,-284.840074,-107.499238,-198.727286,-114.402955,31.570348,-67.448536,-72.040357,-25.961259
CarmenSanchez,-272.074516,-414.224695,-226.426228,-329.255831,-20.797877,-128.807061,15.851962,-79.935212,-94.394618,-1.134169
LiamJohnson,-181.962572,-327.440845,-186.194467,245.698781,-221.693143,-3.661270,-88.266561,18.060626,-23.913595,6.566608


### Comparing lexical 'scores' against GPT-4 HEXACO-PI-R 100 scores.

In [3]:
# now load in the scores from the HEXACO-PI-R 100 survey.
from support.hexaco_pi_r import responses_to_scores, scoring, calc_dimension_scores
pir100 = pd.read_csv('data/hexaco-pi-r-responses-GPT-4.csv', index_col=0)
print(f"Read in {len(pir100)} sets of survey responses (questions={pir100.shape[1]}).")
pir100.rename(index={'CalebJThompson':'CalebJ'}, inplace=True)
pir100.rename(index={'JohnathonKRiley':'JohnathonK'}, inplace=True)
pir100.rename(index={'VioletBRichmond':'VioletB'}, inplace=True)
pir100.columns = range(len(pir100.columns))

# map text responses to score, using scoring mechanism (includes reverse scoring)
pir100_scores = responses_to_scores(pir100, scoring)

# then figure out a score for each dimension for each agent.
pir100_dim_scores = calc_dimension_scores(pir100_scores, scoring)
pir100_dim_scores

Read in 310 sets of survey responses (questions=100).


,Honesty-Humility,Emotionality,Extraversion,Agreeableness,Conscientiousness,Openness
DaveThomas,4.125,2.5625,3.125,2.5625,4.9375,3.0
OliverHarris,4.375,3.25,3.75,2.5,4.875,3.6875
DavidHarrison,4.0625,2.0625,3.75,3.125,4.9375,3.375
JerryMcAlister,2.75,2.75,4.3125,2.625,4.5,3.0
GregoryHarrison,3.5625,2.375,3.125,2.625,4.75,3.3125
...,...,...,...,...,...,...
EmmaJohnson,4.0,3.5625,3.625,3.5,3.5,3.5625
AmeliaMartinez,4.375,3.6875,4.125,3.5625,3.875,3.9375
CarmenSanchez,4.25,3.9375,3.5625,3.5,4.625,3.3125
LiamJohnson,4.0,2.9375,3.75,3.375,2.3125,3.375


In [4]:
from scipy.stats import pearsonr

# now select the best 6 factors (semantic similarity with hexaco)
# to match the 6 hexaco factors.
selection_df = df_dimensions.copy()
selection_df = selection_df[['Dishonesty', 'Gendered-emotionality', 
                             'Introversion', 'Disagreeableness', 
                             'Unconscientiousness', 'Unscholarly']]
selection_df.columns = ['Honesty-Humility', 'Emotionality', 
                        'Extraversion', 'Agreeableness', 
                        'Conscientiousness', 'Openness']

results_df = pd.DataFrame(index=selection_df.columns)
results = {}
for dim in selection_df.columns:
    r, p_value = pearsonr(selection_df[dim], pir100_dim_scores[dim])
    results_df.loc[dim, 'r'] = r
    results_df.loc[dim, 'p_value'] = p_value
    print(f"{dim}: r={r:.3f}, p={p_value:.3f}")
    results[dim] = pearsonr(selection_df[dim], pir100_dim_scores[dim])

pp = pd.DataFrame(results).T
pp = pp.round(4)
pp.columns = ['r', 'p']
pp

Honesty-Humility: r=-0.814, p=0.000
Emotionality: r=0.686, p=0.000
Extraversion: r=-0.833, p=0.000
Agreeableness: r=-0.873, p=0.000
Conscientiousness: r=-0.934, p=0.000
Openness: r=-0.447, p=0.000


,r,p
Honesty-Humility,-0.8141,0.0
Emotionality,0.6861,0.0
Extraversion,-0.8331,0.0
Agreeableness,-0.8731,0.0
Conscientiousness,-0.9344,0.0
Openness,-0.4466,0.0


In [5]:
print("top loading (30) only: ")
print("""Honesty-Humility: r=-0.766, p=0.000
Emotionality: r=0.690, p=0.000
Extraversion: r=-0.790, p=0.000
Agreeableness: r=-0.866, p=0.000
Conscientiousness: r=-0.938, p=0.000
Openness: r=-0.446, p=0.000""")

top loading (30) only: 
Honesty-Humility: r=-0.766, p=0.000
Emotionality: r=0.690, p=0.000
Extraversion: r=-0.790, p=0.000
Agreeableness: r=-0.866, p=0.000
Conscientiousness: r=-0.938, p=0.000
Openness: r=-0.446, p=0.000


In [6]:
# now for all hexaco dimensions, across all agent factors, report pearsons!   
results = pd.DataFrame(index=pir100_dim_scores.columns, columns=df_dimensions.columns)
for hexaco_survey in pir100_dim_scores.columns:
    for agents_factor in df_dimensions.columns:
        r, p_value = pearsonr(pir100_dim_scores[hexaco_survey], df_dimensions[agents_factor])
        results.loc[hexaco_survey, agents_factor] = r

pp = pd.DataFrame(results)
pp = pp.round(4)
pp

,Dishonesty,Disagreeableness,Introversion,Unconscientiousness,Unheroic,Unscholarly,Gendered-emotionality,Unsentimentality,Insensitivity,Unartistic
Honesty-Humility,-0.814143,-0.185195,0.182002,-0.052862,0.359309,0.319759,0.019826,-0.49464,-0.466235,0.558512
Emotionality,-0.327795,-0.583673,-0.57309,0.350056,0.097236,-0.043447,0.686088,-0.527681,-0.659577,-0.16475
Extraversion,0.150988,-0.152745,-0.833114,0.339103,-0.674952,0.051593,0.226523,0.109329,0.30394,-0.511247
Agreeableness,-0.172876,-0.873091,-0.356763,0.43651,0.233616,-0.045804,0.349993,-0.509141,-0.112231,0.00505
Conscientiousness,0.101188,0.446421,0.384238,-0.934436,0.245543,-0.290447,-0.195238,0.189754,0.082279,-0.048806
Openness,0.026385,-0.328707,-0.140307,0.339341,-0.043974,-0.446625,0.377875,-0.309409,-0.350342,-0.385636


### Now same for Llama 3.2's responses

In [7]:
# now load in the scores from the HEXACO-PI-R 100 survey.
from support.hexaco_pi_r import responses_to_scores, scoring, calc_dimension_scores
pir100 = pd.read_csv('data/hexaco-pi-r-responses-llama3.2.csv', index_col=0)
print(f"Read in {len(pir100)} sets of survey responses (questions={pir100.shape[1]}).")
pir100.rename(index={'CalebJThompson':'CalebJ'}, inplace=True)
pir100.rename(index={'JohnathonKRiley':'JohnathonK'}, inplace=True)
pir100.rename(index={'VioletBRichmond':'VioletB'}, inplace=True)
pir100.columns = range(len(pir100.columns))

# map text responses to score, using scoring mechanism (includes reverse scoring)
pir100_scores = responses_to_scores(pir100, scoring)

# then figure out a score for each dimension for each agent.
pir100_dim_scores = calc_dimension_scores(pir100_scores, scoring)
pir100_dim_scores

pir100_dim_scores = pd.read_csv('data/results.csv', index_col=0)
pir100_dim_scores.rename(index={'CalebJThompson':'CalebJ'}, inplace=True)
pir100_dim_scores.rename(index={'JohnathonKRiley':'JohnathonK'}, inplace=True)
pir100_dim_scores.rename(index={'VioletBRichmond':'VioletB'}, inplace=True)
pir100_dim_scores[:5]


Read in 310 sets of survey responses (questions=100).


,Honesty-Humility,Emotionality,Extraversion,Agreeableness,Conscientiousness,Openness
DaveThomas,3.5625,2.8750,2.6250,2.9375,3.4375,3.4375
OliverHarris,3.5625,3.2500,2.6875,3.1875,3.2500,2.8750
DavidHarrison,3.5625,2.9375,2.9375,3.1875,3.2500,3.1250
JerryMcAlister,3.5625,3.0000,2.9375,2.9375,2.8750,3.0625
GregoryHarrison,3.5625,3.0000,3.3125,2.6875,3.1875,3.3750


In [8]:
from scipy.stats import pearsonr

# now select the best 6 factors (semantic similarity with hexaco)
# to match the 6 hexaco factors.
selection_df = df_dimensions.copy()
selection_df = selection_df[['Dishonesty', 'Gendered-emotionality', 
                             'Introversion', 'Disagreeableness', 
                             'Unconscientiousness', 'Unscholarly']]
selection_df.columns = ['Honesty-Humility', 'Emotionality', 
                        'Extraversion', 'Agreeableness', 
                        'Conscientiousness', 'Openness']

results_df = pd.DataFrame(index=selection_df.columns)
results = {}
for dim in selection_df.columns:
    r, p_value = pearsonr(selection_df[dim], pir100_dim_scores[dim])
    results_df.loc[dim, 'r'] = r
    results_df.loc[dim, 'p_value'] = p_value
    print(f"{dim}: r={r:.3f}, p={p_value:.3f}")
    results[dim] = pearsonr(selection_df[dim], pir100_dim_scores[dim])

pp = pd.DataFrame(results).T
pp = pp.round(4)
pp.columns = ['r', 'p']
pp

Honesty-Humility: r=-0.214, p=0.000
Emotionality: r=0.117, p=0.039
Extraversion: r=-0.231, p=0.000
Agreeableness: r=-0.360, p=0.000
Conscientiousness: r=-0.368, p=0.000
Openness: r=-0.013, p=0.819


,r,p
Honesty-Humility,-0.2144,0.0001
Emotionality,0.1172,0.0392
Extraversion,-0.2306,0.0000
Agreeableness,-0.3599,0.0000
Conscientiousness,-0.3685,0.0000
Openness,-0.0130,0.8192


In [9]:
print("results from top 30 only:")
print("""
Honesty-Humility: r=-0.196, p=0.001
Emotionality: r=0.099, p=0.083
Extraversion: r=-0.220, p=0.000
Agreeableness: r=-0.339, p=0.000
Conscientiousness: r=-0.360, p=0.000
Openness: r=-0.014, p=0.800""")

results from top 30 only:

Honesty-Humility: r=-0.196, p=0.001
Emotionality: r=0.099, p=0.083
Extraversion: r=-0.220, p=0.000
Agreeableness: r=-0.339, p=0.000
Conscientiousness: r=-0.360, p=0.000
Openness: r=-0.014, p=0.800


In [10]:
# now for all hexaco dimensions, across all agent factors, report pearsons!   
results = pd.DataFrame(index=pir100_dim_scores.columns, columns=df_dimensions.columns)
for hexaco_survey in pir100_dim_scores.columns:
    for agents_factor in df_dimensions.columns:
        r, p_value = pearsonr(pir100_dim_scores[hexaco_survey], df_dimensions[agents_factor])
        results.loc[hexaco_survey, agents_factor] = r

pp = pd.DataFrame(results)
pp = pp.round(4)
pp

,Dishonesty,Disagreeableness,Introversion,Unconscientiousness,Unheroic,Unscholarly,Gendered-emotionality,Unsentimentality,Insensitivity,Unartistic
Honesty-Humility,-0.214357,-0.110204,0.007255,0.017092,0.032869,0.163052,-0.061103,-0.062906,-0.011083,0.205453
Emotionality,-0.055915,-0.174007,-0.173988,0.224326,-0.132946,0.036461,0.117211,-0.037149,-0.108537,-0.00944
Extraversion,-0.011063,-0.076727,-0.23058,0.154875,-0.149152,0.07188,0.088826,0.025392,0.078512,-0.136777
Agreeableness,-0.086384,-0.359877,-0.189529,0.242797,0.10622,0.053003,0.168958,-0.249708,-0.064039,-0.004006
Conscientiousness,-0.080488,0.251874,0.221197,-0.368485,-0.031294,-0.017189,-0.30589,0.25926,0.035057,-0.027231
Openness,-0.212119,-0.108534,-0.037287,0.116693,-0.002498,-0.013032,0.039562,-0.095015,-0.120331,-0.041491
